### Converting raw data to processed data

In [86]:
import pandas as pd 
import numpy as np 
import os

from utils import * 

def forecasting():

    # reading data
    df_2012_summer = pd.DataFrame(pd.read_csv(os.path.join("/kaggle", Y_2012, "summer.csv")))
    df_2012_winter = pd.DataFrame(pd.read_csv(os.path.join("/kaggle", Y_2012, "winter.csv")))
    df_2012 = pd.concat([df_2012_summer, df_2012_winter])
    df_2012 = df_2012.drop('Athlete', axis=1)
    df_2016 = pd.DataFrame(pd.read_csv(os.path.join("/kaggle", Y_2016, "athletes.csv")))
    df_2020 = pd.DataFrame(pd.read_csv(os.path.join("/kaggle", Y_2020, "medals.csv")))

    # 2020 data changes
    df_2020['gold_medal']=df_2020['medal_code'].apply(lambda x: 1 if x==1 else 0)
    df_2020['silver_medal']=df_2020['medal_code'].apply(lambda x: 1 if x==2 else 0)
    df_2020['bronze_medal']=df_2020['medal_code'].apply(lambda x: 1 if x==3 else 0)
    df_2020 = df_2020[['country_code', 'gold_medal', 'silver_medal', 'bronze_medal']]
    df_2020 = df_2020.groupby('country_code').agg(sum)
    df_2020 = df_2020.reset_index(drop=False)
    df_2020.insert(0, 'year', 2020)
    df_2020['total'] = df_2020['gold_medal'] + df_2020['silver_medal'] + df_2020['bronze_medal']

    # 2016 data changes
    df_2016 = df_2016.rename({'nationality': 'country_code', 'gold': 'gold_medal', 'silver': 'silver_medal', 'bronze': 'bronze_medal'}, axis=1)
    df_2016 = df_2016[['country_code', 'gold_medal', 'silver_medal', 'bronze_medal']]
    df_2016 = df_2016.groupby('country_code').agg(sum)
    df_2016 = df_2016.reset_index(drop=False)
    df_2016.insert(0, 'year', 2016)
    df_2016['total'] = df_2016['gold_medal'] + df_2016['silver_medal'] + df_2016['bronze_medal']

    # 2012 data changes
    years = df_2012[df_2012.Year >= 1948].Year.unique()

    df_2012['gold_medal']=df_2012['Medal'].apply(lambda x: 1 if x=='Gold' else 0)
    df_2012['silver_medal']=df_2012['Medal'].apply(lambda x: 1 if x=='Silver' else 0)
    df_2012['bronze_medal']=df_2012['Medal'].apply(lambda x: 1 if x=='Bronze' else 0)
    df_2012 = df_2012.rename({'Country': 'country_code'}, axis=1)
    df_2012 = df_2012[['Year', 'country_code', 'gold_medal', 'silver_medal', 'bronze_medal']]

    df_temp= pd.DataFrame()
    for year in years:
        temp = df_2012[df_2012.Year == year][['country_code', 'gold_medal', 'silver_medal', 'bronze_medal']]
        temp = temp.groupby('country_code').agg(sum)
        temp = temp.reset_index(drop=False)
        temp.insert(0, 'year', year)
        temp['total'] = temp['gold_medal'] + temp['silver_medal'] + temp['bronze_medal']
        df_temp = pd.concat([df_temp, temp])

    # concatinating all the dataframes
    df = pd.concat([df_temp, df_2016, df_2020], axis=0)
    
    return df

if __name__ == "__main__":
    print(forecasting())

    year country_code  gold_medal  silver_medal  bronze_medal  total
0   1948          ARG           3             8             1     12
1   1948          AUS           2             9             5     16
2   1948          AUT           2             3             7     12
3   1948          BEL           6             6             8     20
4   1948          BRA           0             0            12     12
..   ...          ...         ...           ...           ...    ...
88  2020          UGA           2             1             1      4
89  2020          UKR           1             7            21     29
90  2020          USA         112           108            75    295
91  2020          UZB           3             0             2      5
92  2020          VEN           1             3             0      4

[1404 rows x 6 columns]


### Top 150 countries

In [87]:
df = pd.DataFrame(forecasting())

temp = df.groupby('country_code').agg(sum)
temp = temp[['gold_medal', 'silver_medal', 'bronze_medal', 'total']]
temp.head()

temp = temp.sort_values(['gold_medal', 'silver_medal', 'bronze_medal'], ascending= False)
temp = temp.reset_index(drop=False)

countries_to_consider = temp['country_code'].head(150)
countries_to_consider =list(countries_to_consider)
countries_to_consider
df = df[df['country_code'].isin(countries_to_consider)]

## Visualization on world map

In [88]:
import folium
import geopandas as gpd
import pandas as pd

url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
country_shapes = f"{url}/world-countries.json"
global_polygon = gpd.read_file(country_shapes)

global_polygon.id =  global_polygon.id.apply(lambda x: 'DEN' if x=='DNK' else x)
global_polygon.id =  global_polygon.id.apply(lambda x: 'IRI' if x=='IRN' else x)
global_polygon.id =  global_polygon.id.apply(lambda x: 'NED' if x=='NLD' else x)
global_polygon.id =  global_polygon.id.apply(lambda x: 'POR' if x=='PRT' else x)
global_polygon.id =  global_polygon.id.apply(lambda x: 'RSA' if x=='ZAF' else x)

global_polygon

,id,name,geometry
0,AFG,Afghanistan,"POLYGON ((61.21082 35.65007, 62.23065 35.27066..."
1,AGO,Angola,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6..."
2,ALB,Albania,"POLYGON ((20.59025 41.85540, 20.46317 41.51509..."
3,ARE,United Arab Emirates,"POLYGON ((51.57952 24.24550, 51.75744 24.29407..."
4,ARG,Argentina,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000..."
...,...,...,...
172,PSE,West Bank,"POLYGON ((35.54566 32.39399, 35.54525 31.78251..."
173,YEM,Yemen,"POLYGON ((53.10857 16.65105, 52.38521 16.38241..."
174,RSA,South Africa,"POLYGON ((31.52100 -29.25739, 31.32556 -29.401..."
175,ZMB,Zambia,"POLYGON ((32.75937 -9.23060, 33.23139 -9.67672..."


In [89]:
df['id'] = df['country_code']
df = df.merge(global_polygon, on='id', how='left')

In [90]:
df[df.country_code == 'IRI']

,year,country_code,gold_medal,silver_medal,bronze_medal,total,id,name,geometry
15,1948,IRI,0,0,1,1,IRI,Iran,"POLYGON ((53.92160 37.19892, 54.80030 37.39242..."
55,1952,IRI,0,3,4,7,IRI,Iran,"POLYGON ((53.92160 37.19892, 54.80030 37.39242..."
98,1956,IRI,2,2,1,5,IRI,Iran,"POLYGON ((53.92160 37.19892, 54.80030 37.39242..."
139,1960,IRI,0,1,3,4,IRI,Iran,"POLYGON ((53.92160 37.19892, 54.80030 37.39242..."
181,1964,IRI,0,0,2,2,IRI,Iran,"POLYGON ((53.92160 37.19892, 54.80030 37.39242..."
226,1968,IRI,2,1,2,5,IRI,Iran,"POLYGON ((53.92160 37.19892, 54.80030 37.39242..."
272,1972,IRI,0,2,1,3,IRI,Iran,"POLYGON ((53.92160 37.19892, 54.80030 37.39242..."
315,1976,IRI,0,1,1,2,IRI,Iran,"POLYGON ((53.92160 37.19892, 54.80030 37.39242..."
457,1988,IRI,0,1,0,1,IRI,Iran,"POLYGON ((53.92160 37.19892, 54.80030 37.39242..."
514,1992,IRI,0,1,2,3,IRI,Iran,"POLYGON ((53.92160 37.19892, 54.80030 37.39242..."


In [95]:
df[df.geometry == None]['country_code'].value_counts()

SUI    25
BUL    20
GER    15
GRE    15
YUG    13
       ..
SCG     1
IOP     1
EUN     1
BWI     1
ROC     1
Name: country_code, Length: 61, dtype: int64

In [81]:
global_polygon[global_polygon.name =='South Africa']

,id,name,geometry
174,ZAF,South Africa,"POLYGON ((31.52100 -29.25739, 31.32556 -29.401..."


In [64]:
# df.to_csv('df.csv')

In [8]:


# m = folium.Map(location=[48, -102], zoom_start=3)

# folium.Choropleth(
#     geo_data=state_geo,
#     name="choropleth",
#     data=state_data,
#     columns=["State", "Unemployment"],
#     key_on="feature.id",
#     fill_color="YlGn",
#     fill_opacity=0.9,
#     line_opacity=0.9,
#     legend_name="Unemployment Rate (%)",
# ).add_to(m)

# folium.LayerControl().add_to(m)

# m

,year,country_code,gold_medal,silver_medal,bronze_medal,total
0,1948,ARG,3,8,1,12
1,1948,AUS,2,9,5,16
2,1948,AUT,2,3,7,12
3,1948,BEL,6,6,8,20
4,1948,BRA,0,0,12,12
...,...,...,...,...,...,...
88,2020,UGA,2,1,1,4
89,2020,UKR,1,7,21,29
90,2020,USA,112,108,75,295
91,2020,UZB,3,0,2,5


In [82]:
# import gc

# del df
# gc.collect

<function gc.collect(generation=2)>